### Infêrência Estatística
# Título: Avaliação do impacto da expressões multipalavras nos modelos de linguagem

Este script é similar ao caso base. Mas antes de criar o modelo de linguagem, identificamos as possíveis expressões multipalavras e substituímos a expressão por um único token.

In [1]:
import markovify
import os
import time
import json
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import MWETokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
import math
import numpy as np

Baseamos o código no Read me da biblioteca "Markovify" em: https://github.com/jsvine/markovify/blob/master/README.md

Primeiro criamos uma função para preprocessar o texto.

In [2]:
# Primeiro, criamos uma função para preprocessar os textos
def preprocess(text):
    # Decodificar usando ISO 8859-1 
    text = text.decode('ISO 8859-1')
    # Colocar todas as letras em minúscula
    text = text.lower()
    # Separar as sentenças pelos pontos de finais
    text = text.split('.')
    new_text = []
    for sent in text:
        # Eliminamos as sentenças com menos de 100 caracteres
        if len(sent)> 100:
            # Excluir os espaços no início e fim das sentenças
            new_text.append(sent.strip())
    return(new_text)

Unir os textos em um corpus único para identificar a frequência das palavras e dos bigrams

In [3]:
# Contador de tempo
start_time = time.time()

for (dirpath, _, filenames) in os.walk("Originais/treino"):
    for filename in filenames:
        # Dicionário para armazenar a frequência da palavras
        freq_words = {}
        freq_bigram = {}
        
        # Ler o arquivo e recuperar a frequência das palavras e dos bibrams
        with open(os.path.join(dirpath, filename), 'rb') as f:
            # Preprocessar o texto lido
            text = preprocess(f.read())
            # Iterando por cada sentença
            for sent in text:
                # tokenizando as sentenças
                sent = nltk.word_tokenize(sent)
                # Recuperando a frequência das palavras da sentença
                freq = FreqDist(sent)
                # Atualizando o dicionário com frequência das palavras
                for i in freq:
                    try:
                        freq_words[i] = freq_words[i] + freq[i]
                    except:
                        freq_words[i] = freq[i]
                # Recuperando os bigram
                bigram = nltk.bigrams(sent)
                for i in bigram:
                    try:
                        freq_bigram[i] = freq_bigram[i] + 1
                    except:
                        freq_bigram[i] = 1      
                        
# Fim do contador de tempo
print ("--- %s seconds ---" % (time.time() - start_time))

--- 111.9474127292633 seconds ---


Com a informação das frequências das palavras e dos bigram, é possível calcular a informação mútua.

In [4]:
# Calculando a Informação Mútua para cada bigram
bigram_MI = {}
n_bigram = len(freq_bigram)
n_word = len(freq_words)
for bi in freq_bigram:
    bigram_MI[bi] = math.log((freq_bigram[bi]/n_bigram)/
                             ((freq_words[bi[0]]/n_bigram)*(freq_words[bi[1]]/n_bigram)))

In [5]:
# Imprimindo o número total de bigrams encontrado
print (len(bigram_MI))

11418


Vamos considerar que um bigram é uma MWE aqueles pares de palavras cuja Informação Mútua ultrapassar um determinado limite.

In [6]:
# Criar uma lista de MWE com os bigrams que ultrapassem o limite assinalado na variável "MI".
mwe = {}
MI = 6.0
for bi in bigram_MI:
    if bigram_MI[bi] > MI:
        mwe[bi] = bigram_MI[bi]
print(len(mwe))
for e in mwe:
    print(bigram_MI[e], e[0], e[1])

1302
6.164892506176259  q u í m i c o s   t ó x i c o s 
7.733508424090104  p r o v o c a r   r i s c o s 
7.733508424090104  s a ú d e   h u m a n a 
6.164892506176259  u s o   r e s i d e n c i a l 
6.047109470519875  á r e a s   c o n t a m i n a d a s 
9.342946336524204  m a n c h a   u r b a n a 
7.040361243530159  c o n t a m i n a ç õ e s   p r o v o c a m 
6.858039686736205  p r o v o c a m   r i s c o 
9.342946336524204  a n t e r i o r   v o c a ç ã o 
9.342946336524204  v o c a ç ã o   i n d u s t r i a l 
7.145721759187985  s e t o r   r e s i d e n c i a l 
8.244334047856094  r e s i d e n c i a l   p o s s a m 
9.342946336524204  p o s s a m   c u m p r i r 
6.047109470519875  c u m p r i r   s u a 
6.251903883165888  f u n ç ã o   s o c i a l 
8.649799155964258  s i t u a ç õ e s   p r o v o c a d a s 
8.649799155964258  g e r a m   c o n f l i t o s 
7.397036187468891  e s t a d o   l e g i s l a r 
8.649799155964258  e s s e s   c o n f l i t o s 
8.649799155964258  c 

7.733508424090104  p o p u l a ç õ e s   p r o v a v e l m e n t e 
6.634896135421995  a i n d a   o c o r r e r ã o 
6.251903883165888  t é c n i c a s   e f i c a z e s 
6.634896135421995  r e m e d i a ç ã o   c o n t i n u a r á 
8.649799155964258  c o n t i n u a r á   c o n t r i b u i n d o 
7.733508424090104  s a ú d e   p ú b l i c a 
8.649799155964258  c a r a c t e r i z a ç ã o   f í s i c o - q u í m i c a 
8.649799155964258  c o m b u s t í v e l   d e r i v a d o 
8.244334047856094  c o n s t i t u í d o   b a s i c a m e n t e 
7.145721759187985  3 8   á t o m o s 
7.397036187468891  c a r a c t e r í s t i c a s   b á s i c a s 
8.649799155964258  b á s i c a s   d e s c r i t a s 
6.347214062970213  p e t r o b r á s   a p r e s e n t a m 
7.551186867296149  p r o d u t o   i n f l a m á v e l 
9.342946336524204  m e d i a n a m e n t e   t ó x i c o 
8.244334047856094  o d o r   f o r t e 
8.649799155964258  e s p e c í f i c o   d e p e n d e r á 
6.298423898800781 

7.040361243530159  p o r ç ã o   h i d r o f ó b i c a 
7.040361243530159  p o r ç ã o   h i d r o f í l i c a 
6.8862105637029005  p o r ç ã o   a p o l a r 
6.010741826349001  i ô n i c o s   u t i l i z a d o s 
7.956651975404314  u t i l i z a d o s   c o m e r c i a l m e n t e 
8.649799155964258  c o m e r c i a l m e n t e   i n c l u e m 
8.649799155964258  i n c l u e m   é s t e r e s 
9.342946336524204  é s t e r e s   s u l f a t a d o s 
7.733508424090104  á c i d o s   g r a x o s 
8.649799155964258  p r o d u z i d o s   q u i m i c a m e n t e 
7.145721759187985  e s t r u t u r a   b á s i c a 
6.347214062970213  p o l a r e s   a o s 
7.397036187468891  a g r e g a m   f o r m a n d o 
6.010741826349001  f o r m a n d o   m i c e l a s 
8.649799155964258  m i c e l a r   c r í t i c a 
7.551186867296149  c a d e i a s   a p o l a r e s 
6.858039686736205  g r u p o s   p o l a r e s 
8.244334047856094  f o r m a m   p o n t e s 
6.010741826349001  d i s t r i b u i r 

7.956651975404314  p r o d u t o s   l i b e r a d o s 
7.956651975404314  l i b e r a d o s   p e l o s 
6.45257457862804  p e l o s   m i c r o r g a n i s m o s 
7.733508424090104  c o m u n i d a d e   c i e n t i f i c a 
6.858039686736205  g e r a l   a c e i t a 
8.244334047856094  n e n h u m a   e s p é c i e 
7.956651975404314  m i c r o o r g a n i s m o   d e g r a d a 
8.649799155964258  d e g r a d a   c o m p l e t a m e n t e 
8.649799155964258  c o m p l e t a m e n t e   n e n h u m 
6.45257457862804  m e t a b ó l i c a s   a p r e s e n t a d a s 
7.145721759187985  a p r e s e n t a d a s   o c o r r e m 
6.570357614284423  m ú l t i p l o s   a c e p t o r e s 
7.956651975404314  a c e p t o r e s   f i n a i s 
6.703889006908946  m i c r o o r g a n i s m o s   u s a m 
7.551186867296149  a c e p t o r   t e r m o d i n a m i c a m e n t e 
6.45257457862804  f a v o r á v e l   a t é 
6.298423898800781  e s t e   t e n h a 
6.047109470519875  t e n h a   s u a 
8

7.733508424090104  t e r   o c o r r i d o 
7.733508424090104  n e s t e   ú l t i m o 
6.703889006908946  ú l t i m o   c a s o 
6.45257457862804  b i o t a   d e g r a d a 
7.551186867296149  d e g r a d a   p r e f e r e n c i a l m e n t e 
9.342946336524204  p o l y c h l o r i d e   b i p h e n y l 
8.649799155964258  p s e u d o m o n a s   s p 
7.733508424090104  a n i ô n i c o s   e s t i m u l a r a m 
6.010741826349001  a c e n t u a d a   n a s 
7.551186867296149  c o l u n a   p r o p o r c i o n o u 
6.634896135421995  v i a   s o l u b i l i z a ç ã o 
6.298423898800782  b a i x o   p e s o 
8.649799155964258  s a p o g e n a t   t - 3 0 0 
6.347214062970213  2 0 0   h o r a s 
9.342946336524204  h p a s   p a s s a r a m 
7.145721759187985  c l a r a m e n t e   d e m o n s t r o u 
6.047109470519875  s u a   e f i c á c i a 
9.342946336524204  a k o p a l   n - 3 0 0 
7.263504794844368  1 9 9 7   c h o 
7.145721759187985  c o m p a c t a d o s   d e m o n s t r a r a 

6.075280347486572  c o 2   p r o d u z i d a 
6.45257457862804  m a n t e v e   a t é 
6.298423898800782  d e c i d i u - s e   i n v e s t i g a r 
6.398507357357764  c o m p o r t a m e n t o   a n ô m a l o 
8.649799155964258  a n ô m a l o   o b s e r v a d o 
6.517113099765611  p r i m e i r a s   s e m a n a s 
6.703889006908946  r e p e t i r i a   c a s o 
6.703889006908946  c a s o   f o s s e m 
7.733508424090104  f o s s e m   a d i c i o n a d o s 
8.244334047856094  g á s   c a r b ô n i c o 
8.649799155964258  a v a l i a d a   s e m a n a l m e n t e 
9.342946336524204  p a d r o n i z a d o   c o n t r a 
8.649799155964258  c o n t r a   c a r b o n a t o 
7.040361243530159  p a d r ã o   p r i m á r i o 
6.298423898800781  c o 2   c o n s o m e 
6.777996979062667  c o n s o m e   d u a s 
6.010741826349001  r e a l i z a d o   c o n f o r m e 
7.551186867296149  *   f h c l 
7.551186867296149  f h c l   * 
8.649799155964258  1 0 0 0   * 2 
6.703889006908946  h c l   g 

In [7]:
# Função que recebe um sentença e o dicionário com MWE e retorna a sentença com os MWE unidos por "_". 

def uni_MWE(sent, mwe):
    
    mwe_tokenizer = MWETokenizer(list(mwe.keys()))
    sent = nltk.word_tokenize(sent)
    sent = mwe_tokenizer.tokenize(sent)
    sent = TreebankWordDetokenizer().detokenize(sent)
    return (sent)

Para cada arquivos do diretórios especificado lemos e preprocessamos o texto, unimos os bigrams considerados MWE, criamos uma rede de markov com os n-grams (state_size) e combinamos os modelos de todos os arquivos.

In [8]:
# Contador de tempo
start_time = time.time()

# Cadeia de Markov combinada
combined_model = None

# Contagem de arquivos
n_files = 0

# Iterar pelos diretórios e arquivos
for (dirpath, _, filenames) in os.walk("Originais/treino"):
    for filename in filenames:
        n_files += 1
        
        try:
            # Ler o arquivo e criar uma cadeia de Markov para cada arquivo
            with open(os.path.join(dirpath, filename), 'rb') as f:
                
                # Preprocessar o texto lido
                text = preprocess(f.read())
                text_mwe = []
                
                # Unir as MWE
                for sent in text:
                    text_mwe.append(uni_MWE(sent, mwe))
                
                # Treinar o modelos usando as o texto preprocessado e com MWE
                model = markovify.NewlineText(text_mwe, retain_original=False, state_size=2)

                # Combinar modelo criado para um arquivo com a Cadeia de Markov combinada
                if combined_model:
                    combined_model = markovify.combine(models=[combined_model, model])
                else:
                    combined_model = model
        except:
            print("Não foi possível ler o arquivo", filename)
            #pass
                    
        # Contador de arquivos processados
        if n_files % 50 == 0:
            print ("--- %s arquivos processados ---" % n_files, "--- %s seconds ---" % (time.time() - start_time))
    
# Fim do contador de tempo
print ("--- %s seconds ---" % (time.time() - start_time))

--- 50 arquivos processados --- --- 321.5918276309967 seconds ---
--- 100 arquivos processados --- --- 521.0643892288208 seconds ---
--- 673.9112827777863 seconds ---


In [9]:
# Imprimir um exemplo de sentença aleatória
print(combined_model.make_sentence(max_words=50))

   e x e m p l i f i c a n d o ,    u m   i n o v a d o r   l i m i t e   f i x a d o   p e l a   r e g u l a m e n t a ç ã o   n o r u e g u e s a   d i z   r e s p e i t o   a o   r e n d i m e n t o   d e   s e m e n t e s 


In [10]:
# Gravar modelo em JSON em disco
model_json = combined_model.to_json()
with open('MarkovChainModel_mwe_MI6.json', 'w') as f:
    json.dump(model_json, f)

In [11]:
# Carregar modelo em JSON do disco
with open('MarkovChainModel_mwe_MI6.json', 'r') as f:
    model_json = markovify.Text.from_json(json.load(f))

Como os textos usados são em português, foi necessário codificar em 'ISO 8859-1', já o modelo JSON está codificado em UTF16. Portanto, ao ler e consultar os termos do modelo é necessário usar a codificação correta.

In [12]:
# Geração de sentença aleatória que iniciam com n-palavras

# Palavras que serãao usadas como semente
palavra_1 = 'a'
palavra_2 = 'petrobras'

# Ajustando a codificação para ser usado no modelo
palavra_1 = ' ' + palavra_1 + ' '
palavra_2 = ' ' + palavra_2
palavra_1 = palavra_1.encode('utf16')[3:]
palavra_2 = palavra_2.encode('utf16')[4:]
seed = palavra_1 + palavra_2

#Criando sentença aleatória iniciando com as palavras sementes
print(combined_model.make_sentence_with_start(seed.decode()))

 a   p e t r o b r a s   s e   e n q u a d r a 


Do modelo criado pela biblioteca Markovify, extraímos a cadeia de Markov e gravamos em um dicionário

In [13]:
# Extrair cadeia de Markov do modelo
chain = combined_model.to_dict()['chain']
# Transformando a cadeia em um dicionário 
dic = {}
for state in json.loads(chain):
    dic[(state[0][0],state[0][1])] = state[1]

Criamos uma função para consultar a cadeia de Markov. Dado duas palavras, extraímos as próximas palavras possíveis, a contagem delas no conjunto de treinamento e a soma total. Dessa forma conseguimos calcular  $p(x_3|x_2,x_1)$.

In [14]:
# Função que dado duas palavras, retorna um dicionários com as probabilidades das palavras seguintes
def query_chain(palavra_1, palavra_2):
    # Selecionar o dicionário cuja entrada é o par de palavras "palavra_1" e "palavra_2"
    d = dic[(palavra_1, palavra_2)]
    new_d = {}
    # Transformar a contagem de palavras em probabilidade
    for w in d:
        new_d[w] = d[w]/sum(d.values())
    return (new_d)

O calculamos a perplexidade para cada sentença. Dada as primeiras palavras, obtivemos do modelo a probabilidade da palavra seguinte. Em seguida, avançamos pela sentença recuperando as probablidades.

In [15]:
# Dada uma sentença, a função retorna uma lista com as entropias calculada para cada trio de palavras
def entropy(sent):
    # Primeiro unimos os MWE
    sent = uni_MWE(sent, mwe)
    # Quebra a sentença em tokens
    words = nltk.word_tokenize(sent)
    # Variável para guardar a entropia de cada conjunto de palavras
    ln_prob_Xi = []
    # Janela de iteração pela sentença
    for w in range(len(words)-2):
        # Cálculo a entropia
        try:
            ln_prob_Xi.append(- math.log(query_chain(words[w], words[w+1])[words[w+2]]))
        except:
            pass

    return(np.mean(ln_prob_Xi))

In [16]:
# Função que recebe uma string com o caminho e nome da pasta com os arquivos e retorna a perplexidade

def perplexity(dirpath):

    # Início do contador de tempo
    start_time = time.time()
    
    # Variável para armazenar as entropias
    ent = []

    # Contagem de arquivos
    n_files = 0
    # Iterando pelos arquivos da pasta indicada em
    for (dirpath, _, filenames) in os.walk(dirpath):
        
        for filename in filenames:
            n_files += 1
            try:
                # Ler o arquivo e calcular a entropia
                with open(os.path.join(dirpath, filename), 'rb') as f:
                    # Preprocessar o texto lido
                    text = preprocess(f.read())
                    #Calcular a entropia da sentença
                    for sent in text:
                        ent.append(entropy(sent))

            except:
                print("Não foi possível ler o arquivo", filename)
        
                
            # Contador de arquivos processados
            if n_files % 5 == 0:
                print ("--- %s arquivos processados ---" % n_files, "--- %s seconds ---" % (time.time() - start_time))
    
    
    # Transformando a lista de entropias em uma np.array e excluir os valores "nan"
    ent = np.array(ent)
    ent = ent[~np.isnan(ent)]
    
    # Fim do contador de tempo
    print ("--- %s seconds ---" % (time.time() - start_time))
    
    return (math.exp(np.mean(ent)))

In [17]:
perplexity("Originais/teste")

C:\Users\upe2\AppData\Roaming\Python\Python36\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\upe2\AppData\Roaming\Python\Python36\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


--- 5 arquivos processados --- --- 91.36523985862732 seconds ---
--- 10 arquivos processados --- --- 191.51184558868408 seconds ---
--- 257.02381563186646 seconds ---


18.0179059425318